In [24]:
import os
os.chdir('/Users/nagendrak/Desktop')
import pandas as pd
data=pd.read_csv("crossmasterdata.csv", delimiter = ',' , encoding = 'unicode_escape',header=0, engine='python')
data.head(10)

,the_transaction_id,mdl_num_model_r3,dsm_code,sportsgroup_id
0,7-1041-1041-20200217173532-0001-1520,8491593,300358,267
1,7-1041-1041-20200217173532-0001-1520,8515627,X8515627,258
2,7-1041-1041-20200217185632-0001-1543,8303580,12257,258
3,7-1041-1041-20200217185632-0001-1543,8303580,12257,257
4,7-1041-1041-20200217191748-0002-486,8386248,158595,486
5,7-1041-1041-20200217191748-0002-486,8484820,187032,486
6,7-1041-1041-20200217193918-0001-1559,8559099,165446,257
7,7-1041-1041-20200217193918-0001-1559,8506243,302706,265
8,7-1041-1041-20200217193918-0001-1559,8529981,302706,265
9,7-1041-1041-20200218085951-0001-1619,8326403,111685,265


In [2]:
import os
os.chdir('/Users/nagendrak/Desktop/oms')

In [3]:
def concatinate(indir='/Users/nagendrak/Desktop/oms', outfile='/Users/nagendrak/Desktop/item_level_dec.csv'):
    os.chdir(indir)
    filelist=glob.glob("*.csv")
    dflist=[]
    for filename in filelist:
        print(filename)
        df=pd.read_excel(filename, header=None)
        dflist.append(df)
    concatdf=pd.concat(dflist, axis=0)  
    concatdf.columns=colnames
    concatdf.to_csv(outfile, index=None)

In [25]:
def num_prod_idx(list_prod, idx):
    """
    Get the value at the index on a list.
    Arg: list_prod (list of products we will associate)
         idx (index we want to access in the list)
    Return:
         prod (the number of the product or -1 if the index is not in the list)
    """
    try:
        prod = list_prod[idx]
    except IndexError:
        prod = -1
    return prod


In [26]:
        item_column='mdl_num_model_r3'
        data.rename(columns={'the_transaction_id': 'transaction', item_column:'item'},
                    inplace=True)
        # encode the transaction id
        transactions = data.transaction.unique()
        inverse_transaction_map = {val:i for i, val in enumerate(transactions)}
        data["transaction"] = data["transaction"].map(inverse_transaction_map)

        # drop raws with nan item
        data = data.dropna(subset=['item'])
        data = data.astype({"item": str})
        
        data = data[['transaction','item','sportsgroup_id']]

In [9]:
import time
import pandas as pd
def predict(data,method,max_items,nb_reco):
    """
    Predict the cross-selling assocation.
    """
    print('Prediction starts')
    print('method=', method)
    start = time.time()
    # CROSS SELLING OR CROSS-CLICKING
    item_column='mdl_num_model_r3'
    reco_df = pd.DataFrame(columns=[item_column+'_ref'] + [str(i)
                                                           for i in range(1, 11)])
    items = data['item'].unique()
    print('There are ', len(items), 'items in the dataset')
    for itera, item in enumerate(items[:max_items]):
        list_prod = []
        dict_prod_proba = dict()
        transaction_with_item = set(data[data['item'].isin([item])]['transaction'])
        data_with_item = data[data['transaction'].isin(transaction_with_item)]      
        # A then B <=> Probability of B knowing A
        n_a = data_with_item.shape[0]
        count_value = data_with_item['item'].value_counts()[1:2 * nb_reco + 1]
        i = 0
        while (len(dict_prod_proba) < nb_reco) & (i < len(count_value)):
                list_prod.append(str(count_value.index[i]))
                n_a_b = count_value[i]
                p_b_knowing_a = n_a_b / n_a
                dict_prod_proba[count_value.index[i]] = p_b_knowing_a
                i += 1
        sorted_prod_by_proba = [i[0] for i in sorted(dict_prod_proba.items(), \
                                                     key=lambda kv: kv[1], reverse=True)]
        if method == 'PROBA':
            list_prod = sorted_prod_by_proba

        reco_dict = {str(i+1):num_prod_idx(list_prod, i) for i in range(10)}
        reco_dict[item_column+'_ref'] = item

        reco_df = reco_df.append(reco_dict, ignore_index=True)
        if itera % 250 == 0:
            print('Reco for {} items computed in {:.2f}'.format(itera+1, time.time()-start))

    print("Prediction done for {} items in {:.2f} s".format(len(reco_df), time.time() - start))

    return reco_df

In [16]:
MAX_ITEMS = None
METHOD='PROB'
NB_RECO = 3
data=data
reco_df = predict(data=data,method=METHOD,max_items=MAX_ITEMS,nb_reco=NB_RECO)


Prediction starts
method= PROB
There are  5588 items in the dataset
Reco for 1 items computed in 0.24
Reco for 251 items computed in 30.63
Reco for 501 items computed in 61.38
Reco for 751 items computed in 91.35
Reco for 1001 items computed in 122.14
Reco for 1251 items computed in 157.06
Reco for 1501 items computed in 191.22
Reco for 1751 items computed in 226.83
Reco for 2001 items computed in 260.57
Reco for 2251 items computed in 295.21
Reco for 2501 items computed in 329.43
Reco for 2751 items computed in 360.26
Reco for 3001 items computed in 395.32
Reco for 3251 items computed in 430.80
Reco for 3501 items computed in 458.44
Reco for 3751 items computed in 481.48
Reco for 4001 items computed in 502.74
Reco for 4251 items computed in 520.94
Reco for 4501 items computed in 539.17
Reco for 4751 items computed in 556.29
Reco for 5001 items computed in 572.17
Reco for 5251 items computed in 587.25
Reco for 5501 items computed in 602.01
Prediction done for 5588 items in 607.35 s


In [18]:
reco_df.to_csv('bundles.csv')


In [27]:
def predict_wfilter(data,method,max_items,nb_reco):
    """
    Predict the cross-selling assocation.
    """
    print('Prediction starts')
    print('method=', method)
    start = time.time()
    item_column='model_code_r3'
    # CROSS SELLING OR CROSS-CLICKING
    reco_df = pd.DataFrame(columns=[item_column+'_ref'] + [str(i)
                                                           for i in range(1, 11)])
    items = data['item'].unique()
    print('There are ', len(items), 'items in the dataset')
    for itera, item in enumerate(items[:max_items]):
        list_prod = []
        dict_prod_proba = dict()
        transaction_with_item = set(data[data['item'].isin([item])]['transaction'])
        df_sportsgroup = data[['item', 'sportsgroup_id']].drop_duplicates()
        dsm_to_sportsgroup = dict(zip(df_sportsgroup['item'],
                                      df_sportsgroup['sportsgroup_id']))
        sportsgroup_current = dsm_to_sportsgroup[item]
        data_with_item = data[data['transaction'].isin(transaction_with_item) & data['sportsgroup_id'].isin([sportsgroup_current])]
        # A then B <=> Probability of B knowing A
        n_a = data_with_item.shape[0]
        count_value = data_with_item['item'].value_counts()[1:2 * nb_reco + 1]
        i = 0
        while (len(dict_prod_proba) < nb_reco) & (i < len(count_value)):
                list_prod.append(str(count_value.index[i]))
                n_a_b = count_value[i]
                p_b_knowing_a = n_a_b / n_a
                dict_prod_proba[count_value.index[i]] = p_b_knowing_a
                i += 1
        sorted_prod_by_proba = [i[0] for i in sorted(dict_prod_proba.items(), \
                                                     key=lambda kv: kv[1], reverse=True)]
        if method == 'PROB':
            list_prod = sorted_prod_by_proba

        reco_dict = {str(i+1):num_prod_idx(list_prod, i) for i in range(10)}
        reco_dict[item_column+'_ref'] = item

        reco_df = reco_df.append(reco_dict, ignore_index=True)
        if itera % 250 == 0:
            print('Reco for {} items computed in {:.2f}'.format(itera+1, time.time()-start))

    print("Prediction done for {} items in {:.2f} s".format(len(reco_df), time.time() - start))
    print(reco_df)

    return reco_df

In [28]:
MAX_ITEMS = 20
METHOD='PROB'
NB_RECO = 10
data=data
reco_df = predict_wfilter(data=data,method=METHOD,max_items=MAX_ITEMS,nb_reco=NB_RECO)


Prediction starts
method= PROB
There are  5588 items in the dataset
Reco for 1 items computed in 0.48
Prediction done for 20 items in 6.47 s
   model_code_r3_ref        1        2        3        4        5        6  \
0            8491593  8491592  8286996  8042506  8546785  8012859  8489600   
1            8515627  8395704  8303580  8533553  8480885  8381098  3796979   
2            8303580  8177653  8204040  8204042  8480885  8328338  8328337   
3            8386248  8484096  8386249  8517598  8385672  8517565  8344978   
4            8484820  8484819  8484822  8344978  8576804  8381582  8375469   
5            8559099  8296178  8519871  8351755  8528958  8488035  8554249   
6            8506243  8529981  8388642  8388645  8311896  8510183  8326403   
7            8529981  8506243  8388642  8388645  8510183  8311896  8379446   
8            8326403  8388642  8379446  8388645  8379586  8510183  8491877   
9            8587966  8213655  8394851  8602255  8501381  4733799  8587967   
1

In [23]:
reco_df.to_csv('sport_bundles.csv')
